In [ ]:
import pandas as pd
import copy
import scipy.stats as stats
import time
import numpy as np

In [ ]:
def linearize(value, agg=None):
    if agg is None:
        agg = []
    if isinstance(value, (tuple, list)):
        for item in value:
            linearize(item, agg)
    else:
        agg.insert(0,value)
    return agg[:-1], agg[-1]

In [ ]:
def unnest(d, keys=[]):
    result = []
    for k, v in d.items():
        if isinstance(v, dict):
            result.extend(unnest(v, keys + [k]))
        else:
            result.append(tuple(keys + [k, v]))
    return result

In [ ]:
from collections import defaultdict
from heapq import *

def dijkstra(edges, f, t):
    g = defaultdict(list)
    for l,r,c in edges:
        g[l].append((c,r))
    q, seen, mins = [(0,f,())], set(), {f: 0}
    while q:
        (cost,v1,path) = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            path = (v1, path)
            if v1 == t: return (cost, path)

            for c, v2 in g.get(v1, ()):
                if v2 in seen: continue
                prev = mins.get(v2, None)
                next = cost + c
                if prev is None or next < prev:
                    mins[v2] = next
                    heappush(q, (next, v2, path))

    return float("inf")

In [ ]:
## Currently using Truncated normal distribution which can be changed to accomodate more types of distributions

In [ ]:
import math
def network_construction(graph,max_t, ff_time, draws):
    graphs = [copy.deepcopy(graph) for i in range(draws)]
    for key, value in graph.items():
        for k, v in value.items():
            mu = graph[key][k]
            sigma = 0.2*mu
            weights = stats.truncnorm(((ff_time[key][k]) - mu) / sigma, max_t[key][k], loc=mu, scale=sigma).rvs(size = draws)
            for i in range(draws):
                graphs[i][key][k] = weights[i]
    return graphs

In [ ]:
def fix_dict(d):
    for key in d:
        for x in d[key]:
            for y in d[key][x]:
                d[key][x] = d[key][x][y]
    return d

In [ ]:
def get_paths(tt, max_t, ff_time, origin, desti, draws):
    A = []
    graphs = network_construction(tt,max_t,ff_time, draws)
    for i in range(draws):
        g = graphs[i]
        path = linearize(dijkstra(unnest(g), origin,desti))
        if path[0] not in A:
            A.append(path[0])
        else:
            continue
    return A

In [ ]:
if __name__ == "__main__":
    path = r'C:\Users\n10405992\OneDrive - Queensland University of Technology\Raghav Malhotra - RM\Code\Path Choice\Data\final\Simulation Network.csv'
    network = pd.read_csv(path, usecols = ['From_o_sen', 'To_d_senso','Total_Kilo_x', 'Total_Minu_x', 'speed'])
    network['FFT'] = round((network['Total_Kilo_x']/network['speed'])*3600,2)
    network['Total_Minu_x'] = round(network['Total_Minu_x']*60,2)
    network['Max_time'] = round((network['Total_Kilo_x']/10)*3600,2)
    tt = network.groupby('From_o_sen')[['To_d_senso', 'Total_Minu_x']].apply(lambda x: x.set_index('To_d_senso').to_dict(orient='index')).to_dict()
    fft = network.groupby('From_o_sen')[['To_d_senso', 'FFT']].apply(lambda x: x.set_index('To_d_senso').to_dict(orient='index')).to_dict()
    max_time = network.groupby('From_o_sen')[['To_d_senso', 'Max_time']].apply(lambda x: x.set_index('To_d_senso').to_dict(orient='index')).to_dict()
#     point = df.groupby('OriginOID')[['DestinationOID', 'Total_Distance']].apply(lambda x: x.set_index('DestinationOID').to_dict(orient='index')).to_dict()
    tt = fix_dict(tt)
    fft = fix_dict(fft)
    max_time = fix_dict(max_time)
    origins = [10423,10599]
    desti = [10651,10415]

## Runtime analysis

In [ ]:
origins = [10423,10599]
destinations = [10651,10415]

In [ ]:
runtime = {}
avg_paths = {}
for origin in origins:
    for destination in desti:
        for draws in range(10,151,5):
            start = time.time()
            paths = get_paths(tt, max_time, fft, origin, destination, draws)
            end = time.time()
            num_paths = len(paths)
            try:
                runtime[draws] += (end-start)/4
                avg_paths[draws] += num_paths/4
            except:
                runtime[draws] = (end-start)/4
                avg_paths[draws] = num_paths/4
            new_paths = []
            for path in paths:
                z = []
                for i in range(len(path)-1):
                    z.append(int(str(path[i]) + str(path[i+1])))
                new_paths.append(z)
            output = pd.DataFrame({'ID': range(len(paths)), 'Paths': new_paths})
            output.to_csv(str(origin) + "-" + str(destination) + "\\" + "Draws " + str(draws) + ".csv")

In [ ]:
metrics = pd.DataFrame({'Avg Paths': avg_paths, 'Runtime': runtime}).to_csv("metrics.csv", index = False)

## Output paths

In [ ]:
for origin in origins:
    for destination in destinations:
        new_paths = []
        paths = get_paths(tt, max_time, fft, origin, destination, 50)
        for path in paths:
            z = []
            for i in range(len(path) - 1):
                z.append(int(str(path[i]) + str(path[i+1])))
            new_paths.append(z)
        output = pd.DataFrame({'ID': np.arange(0,len(new_paths)), 'Path': new_paths})
        output.to_csv(str(origin) + "-" + str(destination)+".csv", index = False)